### Naive Bayes solution in Python 3.6
##### Ruihang Du
The following is a Python 3 implementation of Naive Bayes spam detection algorithm.

In [1]:
%%latex
Refresher on Naive Bayes:
\begin{align}
P(y_i \mid w_1, w_2, \dots, w_n) &\propto P(w_1, w_2, \dots, w_n \mid y_i)P(y_i)\\
&= P(w_1 \mid y_i)P(w_2 \mid y_i)\dots P(w_n \mid y_i)P(y_i) &\text{Under the independence assunption}
\end{align}

<IPython.core.display.Latex object>

In [2]:
import pandas as pd

Note that if you are using Python 2, make sure you do the following:
```Python
from __future__ import division, print_function
```

In [3]:
df = pd.read_csv('spam_files\spam_train.csv', encoding = "ISO-8859-1")
data = df[['label', 'text']]

Use the __read_csv__ function in __Pandas__ to create a data frame from a csv file (Pandas is not required but generally makes your life easier dealing with formatted data).

Remember to change the path to the data file accordingly (use slash instead of backslash if you are using Linux).

Then we select only the __"label"__ column and the __"text"__ column from the data frame.

In [4]:
ham_dict = {}
spam_dict = {}

First we initialize the dictionarys to store the __number of occurrences__ of each word in __spam__ and __ham__ emails

In [5]:
spam_df = data.loc[(data.label == 'spam')].reset_index(drop=True)
ham_df = data.loc[(data.label == 'ham')].reset_index(drop=True)
            
for i in range(len(spam_df)):
    # look at every word in the email
    for w in spam_df['text'][i].split(' '):
        if w in spam_dict:
            spam_dict[w] += 1       # update the count of this word
        else:
            spam_dict[w] = 1        # record the first occurrence
            
for i in range(len(ham_df)):
    for w in ham_df['text'][i].split(' '):
        if w in ham_dict:
            ham_dict[w] += 1
        else:
            ham_dict[w] = 1

Now we begin to fill in each dictionary.

The first two lines separate the spam data and ham data to two __different tables__. Then for __each table__ we enumerate all the emails and record the number of occurrences for each single word.

In [6]:
num_spam, num_ham = sum(spam_dict.values()), sum(ham_dict.values())

for key in spam_dict:
    spam_dict[key] /= num_spam
    
for key in ham_dict:
    ham_dict[key] /= num_ham

Normalize the count of words in each dictionary to __probabilities__. 

In [7]:
%%latex
Remember that for each word $w_i$, we need to calculate $P(w_i \mid spam)$ and $P(w_i \mid ham)$, which translates to
"Given that the email is a spam (or ham), how likely is it that I see the word $w_i$?"
\begin{align*}
P(w_i \mid spam) &= \frac{P(w_i, spam)}{P(spam)}\\
&= \frac{\# \text{$w_i$ occurred in spam}}{\# \text{spam}}
\end{align*}

<IPython.core.display.Latex object>

In [8]:
test_df = pd.read_csv('spam_files\spam_test.csv', encoding = "ISO-8859-1")
test_data = df[['label', 'text']]

total = 0
correct = 0
# For each text
for i in range(len(test_data)):
    total += 1
    txt_to_eval = test_data['text'][i]
    # split up the text into words and lookup in the spam_dict
    words = txt_to_eval.split(' ')
    l_s, l_h = 1, 1    # initialize likelihoods
    for w in words:    
        # First evaluate the likelihood of it being a spam
        if w in spam_dict:
            l_s *= spam_dict[w]
        else:
            l_s *= 1/(num_spam + 1)
    
        # Then evaulate the likelihood of it being a ham
        if w in ham_dict:
            l_h *= ham_dict[w]
        else:
            l_h *= 1/(num_ham + 1)
    
    l_s *= num_spam/(num_spam + num_ham)
    l_h *= num_ham/(num_spam + num_ham)
    
    if (l_s > l_h and test_data['label'][i] == 'spam') \
    or (l_s < l_h and test_data['label'][i] == 'ham'):
        correct += 1

print("accuracy: {}".format(correct/total))

accuracy: 0.9034907597535934


After training, we then test our NB classifier on the test set.

For each test email, we calculate the likelihood of it being a spam and that of it being a ham. We then predict the class based on whichever likelihood is greater.